In [16]:
import os
import sys
os.chdir("E:\\2_Studium\\1_Mannheim\\4_Semester\\Masterarbeit\\code\\master-thesis-code\\data")
import pandas as pd

In [68]:
feature_abt_buy_test = pd.read_csv("abt_buy/features_abt_buy_test")
feature_abt_buy_train = pd.read_csv("abt_buy/features_abt_buy_train")
abt_buy_gold_standard_train = pd.read_csv("abt_buy/abt_buy_gold_standard_train.csv",sep=";")
abt_buy_gold_standard_test = pd.read_csv("abt_buy/abt_buy_gold_standard_test.csv",sep=";")

feature_amazon_google_test = pd.read_csv("amazon_google/features_amazon_google_test")
feature_amazon_google_train = pd.read_csv("amazon_google/features_amazon_google_train")

feature_wdc_phones_test = pd.read_csv("wdc_product/features_phones_phones_catalog_test")
feature_wdc_phones_train = pd.read_csv("wdc_product/features_phones_phones_catalog_train")

feature_wdc_headphones_test = pd.read_csv("wdc_product/features_headphones_headphones_catalog_test")
feature_wdc_headphones_train = pd.read_csv("wdc_product/features_headphones_headphones_catalog_train")

feature_dbpedia_viaf_test = pd.read_csv("author/features_DBPediaAuthors_VIAFDataAuthors_test")
feature_dbpedia_viaf_train = pd.read_csv("author/features_DBPediaAuthors_VIAFDataAuthors_train")
gs_dbpedia_viaf_test = pd.read_csv("author/DBPediaAuthors_VIAFDataAuthors_gold_standard_test.csv",sep=";")
gs_dbpedia_viaf_train = pd.read_csv("author/DBPediaAuthors_VIAFDataAuthors_gold_standard_train.csv",sep=";")

feature_dbpedia_wiki_test = pd.read_csv("author/features_DBPediaAuthors_WikiDataAuthors_test")
feature_dbpedia_wiki_train = pd.read_csv("author/features_DBPediaAuthors_WikiDataAuthors_train")

feature_dbpedia_dnb_test = pd.read_csv("author/features_DBPediaAuthors_DnbDataAuthors_test")
feature_dbpedia_dnb_train = pd.read_csv("author/features_DBPediaAuthors_DnbDataAuthors_train")

In [61]:
# Data profiling information
metadata = [
    ['abt_buy',feature_abt_buy_train.shape[1],
    feature_abt_buy_train.shape[0],feature_abt_buy_test.shape[0]],
    ['amazon_google',feature_amazon_google_train.shape[1],
    feature_amazon_google_train.shape[0],feature_amazon_google_test.shape[0]],
    ['wdc_phones',feature_wdc_phones_train.shape[1],
    feature_wdc_phones_train.shape[0],feature_wdc_phones_test.shape[0]],
    ['wdc_headphones',feature_wdc_headphones_train.shape[1],
    feature_wdc_headphones_train.shape[0],feature_wdc_headphones_test.shape[0]],
    ['dbpedia_viaf',feature_dbpedia_viaf_train.shape[1],
    feature_dbpedia_viaf_train.shape[0],feature_dbpedia_viaf_test.shape[0]],
    ['dbpedia_wiki',feature_dbpedia_wiki_train.shape[1],
    feature_dbpedia_wiki_train.shape[0],feature_dbpedia_wiki_test.shape[0]],
    ['dbpedia_dnb',feature_dbpedia_dnb_train.shape[1],
    feature_dbpedia_dnb_train.shape[0],feature_dbpedia_dnb_test.shape[0]],
    ]
df_profiling = pd.DataFrame(metadata, columns=['Dataset', '# Attributes', '# Train', '# Test'])
df_profiling

,Dataset,# Attributes,# Train,# Test
0,abt_buy,19,5730,1432
1,amazon_google,24,6753,1687
2,wdc_phones,135,1762,440
3,wdc_headphones,142,1163,290
4,dbpedia_viaf,27,15316,3828
5,dbpedia_wiki,27,19228,4807
6,dbpedia_dnb,26,13863,3465


# Preprocessing abt_buy

In [38]:
ids_abt_buy_train = feature_abt_buy_train[["source_id","target_id"]]
ids_abt_buy_test = feature_abt_buy_test[["source_id","target_id"]]
abt_original = pd.read_csv("abt_buy/abt.csv",encoding='latin1').add_prefix('left_')
buy_original = pd.read_csv("abt_buy/buy.csv",encoding='latin1').add_prefix('right_')

#train
abt_buy_train = ids_abt_buy_train.merge(
    abt_original,left_on="source_id",right_on="left_subject_id").merge(
    buy_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
abt_buy_train['id'] = abt_buy_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
abt_buy_test = ids_abt_buy_test.merge(
    abt_original,left_on="source_id",right_on="left_subject_id").merge(
    buy_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
abt_buy_test['id'] = abt_buy_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

In [39]:
#train
abt_buy_gold_standard_train['id'] = abt_buy_gold_standard_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
abt_buy_gold_standard_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
abt_buy_gold_standard_test['id'] = abt_buy_gold_standard_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
abt_buy_gold_standard_test.drop(['source_id','target_id'],axis=1,inplace=True)

In [40]:
abt_buy_train = abt_buy_train.merge(abt_buy_gold_standard_train,on='id')
abt_buy_train = abt_buy_train.rename(columns={'matching':'label'})
abt_buy_train.drop('right_manufacturer',axis=1,inplace=True)
abt_buy_train['label'] = abt_buy_train['label'].astype(int)

In [41]:
abt_buy_test = abt_buy_test.merge(abt_buy_gold_standard_test,on='id')
abt_buy_test = abt_buy_test.rename(columns={'matching':'label'})
abt_buy_test.drop('right_manufacturer',axis=1,inplace=True)
abt_buy_test['label'] = abt_buy_test['label'].astype(int)
abt_buy_test.head()

,source_id,target_id,left_name,left_description,left_price,right_name,right_description,right_price,id,label
0,17187,202041204,Universal IR/RF Remote - MX350,Universal IR/RF Remote - MX350/ Controls Up To...,149.95,Universal MX-350 Osiris Remote Control - MX350,"TV, VCR, DVD Player, CD Player, Cable Box, Sat...",93.9,17187-202041204,1
1,17187,203069939,Universal IR/RF Remote - MX350,Universal IR/RF Remote - MX350/ Controls Up To...,149.95,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,17187-203069939,0
2,33965,203069939,Logitech Harmony One Advanced Universal Remote...,Logitech Harmony One Advanced Universal Remote...,249.00,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,33965-203069939,0
3,31273,203069939,Sony Universal Remote Commander Remote Control...,Sony Universal Remote Commander Remote Control...,NaN,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,31273-203069939,0
4,27779,205844283,Sony LCS-CSQ/B Black Soft Carrying Case - LCSCSQB,Sony LCS-CSQ/B Black Soft Carrying Case - LCSC...,NaN,Sony LCS-CSQ Soft Cyber-shot Camera Case - LCS...,Top Loading - Polyamide - Black,NaN,27779-205844283,1


## Split Train into Train and Validation

In [77]:
y = abt_buy_train['label']

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(abt_buy_train, y,
                                                stratify=y, 
                                                test_size=0.25)

In [80]:
abt_buy_train = X_train
abt_buy_validation = X_val

In [81]:
abt_buy_train.to_csv("abt_buy/abt_buy_train",index=False)
abt_buy_validation.to_csv("abt_buy/abt_buy_validation",index=False)
abt_buy_test.to_csv("abt_buy/abt_buy_test",index=False)

# Preprocessing dbpedia_viaf

In [69]:
ids_dbpedia_viaf_train = feature_dbpedia_viaf_train[["source_id","target_id"]]
ids_dbpedia_viaf_test = feature_dbpedia_viaf_test[["source_id","target_id"]]
dbpedia_original = pd.read_csv("author/DBpediaAuthors.csv",encoding='latin1').add_prefix('left_')
viaf_original = pd.read_csv("author/VIAFDataAuthors.csv",encoding='latin1').add_prefix('right_')

#train
dbpedia_viaf_train = ids_dbpedia_viaf_train.merge(
    dbpedia_original,left_on="source_id",right_on="left_subject_id").merge(
    viaf_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
dbpedia_viaf_train['id'] = dbpedia_viaf_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
dbpedia_viaf_test = ids_dbpedia_viaf_test.merge(
    dbpedia_original,left_on="source_id",right_on="left_subject_id").merge(
    viaf_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
dbpedia_viaf_test['id'] = dbpedia_viaf_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

In [70]:
#train
gs_dbpedia_viaf_train['id'] = gs_dbpedia_viaf_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_dbpedia_viaf_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
gs_dbpedia_viaf_test['id'] = gs_dbpedia_viaf_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_dbpedia_viaf_test.drop(['source_id','target_id'],axis=1,inplace=True)

In [71]:
dbpedia_viaf_train = dbpedia_viaf_train.merge(gs_dbpedia_viaf_train,on='id')
dbpedia_viaf_train = dbpedia_viaf_train.rename(columns={'matching':'label'})
dbpedia_viaf_train.drop(['left_abstract','left_links'],axis=1,inplace=True)
dbpedia_viaf_train['label'] = dbpedia_viaf_train['label'].astype(int)

In [73]:
dbpedia_viaf_test = dbpedia_viaf_test.merge(gs_dbpedia_viaf_test,on='id')
dbpedia_viaf_test = dbpedia_viaf_test.rename(columns={'matching':'label'})
dbpedia_viaf_test.drop(['left_abstract','left_links'],axis=1,inplace=True)
dbpedia_viaf_test['label'] = dbpedia_viaf_test['label'].astype(int)

## Split Train into Train and Validation


In [76]:
y = dbpedia_viaf_train['label']
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dbpedia_viaf_train, y,
                                                stratify=y, 
                                                test_size=0.25)
dbpedia_viaf_train = X_train
dbpedia_viaf_validation = X_val
dbpedia_viaf_train.to_csv("author/dbpedia_viaf_train",index=False)
dbpedia_viaf_validation.to_csv("author/dbpedia_viaf_validation",index=False)
dbpedia_viaf_test.to_csv("author/dbpedia_viaf_test",index=False)